# Introduction

Below is a 'clean' version of the Jupyter Notebook used in Integration skills 1 and 2. You can use it if you want to look at water balance and water quality in the ponds.

In your application, consider the following points:
1. Summarise the methods in your PBL report
2. Document all choices you make with respect to parameter settings and processes included (or omitted)
3. Remember that a model is 'just' a set of numerical rules. You have to interpret the results considering meaning, reliability, accuracy
4. Consider which parameters or processes may be most sensitive and/or unreliable and test how sensitive the model is to those. Document the sensitivity as a confidence interval: in which interval are you confident that the model results will be?
5. **NEVER** weaken your report by writing things like: 'Due to uncertainty in x, the model results are uncertain.' In this way you entirely undermine your model results. **Instead**, document how reliable the model is in terms of a confidence interval (see point 4).

## Things to check:
- did you set up the parameters for the evaporation and Q-terms in the "simple case"?
- did you set up the realistic Q-terms in the "realistic case"?

### 1 Load the necessary python packages

In [ ]:
# Load necessary python packages.
import sys
!{sys.executable} -m pip install matplotlib pandas cufflinks > /dev/null; # Remove > /dev/null in case of errors.

from ipywidgets import interact
import numpy as np
from numpy import log as ln
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf

# (Ignore the pip version warning!)

### 2 Set some physical constants that stay the same for all calculations

In [ ]:
# Physical constants           # Set the value of all physical constants here.
Lv     = 2.45e6                # J/kg   Latent heat of vaporisation
gamma  = 0.066                 # kPa/K  Psychrometer constant

# Parameters                   # Set all parameter values here
dt     = 1.                    # h timestep is hourly, because the supporting meteo data are on 1 h timesteps.
A      = 5000.                 # m2
d_max  = 0.                    # m above the surface
d_min  = -3.                   # m above the surface (depth)
V_max  = 10000.     # m3 max Volume of the pond (assuming the surface area does not depend on the level)

# Initial conditions           # Set the initial conditions for the volume
V_ini  = V_max                 # m3 the initial Volume (here: assume pond is full at the start, 
                               # but you can set it to a lower value if you like).

# If the initial volume is bigger than the maximum volume of the pond, set it to be equal to the maximum value.
if V_ini > V_max :
    V_ini = V_max

### 3 Load meteorological data

1. Load the data for simple calculation where evaporation and Q-terms (except for the lateral outflow) are constant. 
2. Load the data for Makkink calculation of evaporation. Q-terms are still constant (except for the lateral outflow).
3. Load the data for realistic calculation where evaporation is calculated using Makkink method and Q-terms are set to some realistic value.
4. Load the dry deposition rate calculated in Basic Skills Air Quality. If you chose not to work with Air Quality part, you can load the default ammonia deposition values by setting outfilename_BS_AQ = 'Data_Output_BasicSkills_AirQuality.csv'. Otherwise, write the name of your saved file.

N.B. Note that here we offer the opportunity to read in the terms from a .csv file.  We will only use the values of P here, the others we will overwrite at the bottom of the cell. 

In [ ]:
# 1.---------------------------------------------------------------------------
# Load meteorological data for for calculation where evaporation is calculated using Makkink method. 
# We read in precipitation (P; mm/h), atmospheric temperature (T_a; degC) and global radiation (Rg; W/m2).
df_meteo  = pd.read_csv('../Data_IntegrationCourseSWA/Data_Meteo_Veenkampen_2013-2019.csv', sep=';', index_col='date-time', 
    usecols=['date-time', 'P', 'Ta', 'Rg'], parse_dates=['date-time'])

# Convert precipitation from mm/h to m/h using the pond's surface area.
df_meteo['P']      = df_meteo['P']*A/1000. 


# 2.--------------------------------------------------------------------------- 
# Load the dry deposition rate calculated in IntegrationCourse_PBL_NH3Fluxes.ipynb
# Load the dry deposition rate for the three stations i.e., Wekerom, Vredepeel and Zegveld
# as F_NH3_W, F_NH3_V, and F_NH3_Z, respectively. 
outfilename_FluxNH3 = '../Data_IntegrationCourseSWA/FluxNH3.csv'  

df_drydepo = pd.read_csv(outfilename_FluxNH3, sep=';', 
    index_col='date-time', usecols=['date-time', 'F_NH3_W', 'F_NH3_V', 'F_NH3_Z'], parse_dates=['date-time'])

In [ ]:
# 3.--------------------------------------------------------------------------- 
# Set evaporation and lateral inflow to have constant values.
df_meteo['E'          ] = 0.5   # m3/h  Set some value for evaporation. 
df_meteo['Ql_in'      ] = 0.1   # m3/h  Set some small value for lateral inflow.
df_meteo['Qk_in'      ] = 0.0   # m3/h  Set the vertical inflow to zero for now.
df_meteo['Qk_out'     ] = 0.0   # m3/h  Set the vertical outflow to zero for now.  

# Add realistic values for Ql_in_real, Qk_in_real and Qk_out_real
df_meteo['Ql_in_real' ] = 1./24. # m3/h Lateral inflow;   ENTER REALISTIC VALUES FOR YOUR GROUP's LANDSCAPE UNIT 
df_meteo['Qk_in_real' ] = 5./24. # m3/h Vertical inflow;  ENTER REALISTIC VALUES FOR YOUR GROUP's LANDSCAPE UNIT 
df_meteo['Qk_out_real'] = 0.0    # m3/h Vertical outflow; ENTER REALISTIC VALUES FOR YOUR GROUP's LANDSCAPE UNIT 


In [ ]:
# Makkink approach for Evaporation

# Calculate evaporation rate based on the Makkink method (E_mak; m^3 h^-1).
es_0               = 0.6107      # kPa
a                  = 7.5         #
b                  = 237.3       # degC
df_meteo['es']     = es_0*10**((a*df_meteo['Ta'])/(b+df_meteo['Ta']))
df_meteo['s']      = df_meteo['es']*ln(10)*a*b    /(b+df_meteo['Ta'])**2
df_meteo['E_ref']  = 0.65*(1/Lv)*(df_meteo['s']/(df_meteo['s']+gamma))*df_meteo['Rg']

# Calculate the evaporation rate. Note that the change of units between the reference evaporation and 
# the actual evaporation rate.
Cx                 = 1.25        # 'Crop' factor for water.
df_meteo['E_mak']  = Cx*df_meteo['E_ref']*3600.*A/1000.


### 4 Define boundary conditions for ammonia concentrations in inflowing water

You measured those in the Basic Skills Acquatic Ecology.

N.B.: Because of the C-situation, you weren't able to collect the measurements, but Edwin Peeters has been so kind to collect them for you. Please see the map on Brightspace --> Integration Course SWA --> Content --> Basic and Integration Skills --> Integration Skills 2 --> Integration Skills 2 Introduction.pdf, page 2.

Note that you need to convert the concentration units from $mg\,l^{-1}$ to $\mu$g N m$^{-3}$.

### 5 Define parameters and limitations for clorofyl calculations

In [ ]:
# Write them in order: concentration in precipitation C_N_P, concentration in lateral inflow C_N_l_in, 
# concentration in vertical inflow C_N_k_in. 
# All concentrations should be written in ug N/m3.
C_N_P     = 0.1e-6    # ug/m3 Concentration of N in Precipitation.   The value should be order of 10^-1 mg/l  = 0.1e-6 ug/m3
C_N_Ql_in = 2.0e-6    # ug/m3 Concentration of N in lateral inflow.  The value should be order of 1 mg/l = 1.0e-6 ug/m3
C_N_Qk_in = 0.0e-6    # ug/m3 Concentration of N in vertical inflow. The value should be order of 1 mg/l = 1.0e-6 ug/m3

# Calculate the wet deposition rate. Note that the precipitation is already converted
# from mm/h to m3/h using the pond's surface area. We did this in the box above.
df_meteo['dNdt_FNH3_wet']   = df_meteo['P'] * C_N_P

# Convert the units of the deposition rate to g/h. Therefore, set the area of the lake 
# (A; m2), convert the seconds to hours, use the ratio 17/14 to convert the dry deposition 
# of NH3 to dry deposition of nitrogen N. 
# Lastly, let's not forget to convert ug to g.
df_drydepo['dNdt_FNH3_dry'] = df_drydepo['F_NH3_W']*A*3600.*17./14.*0.000001  # Use Wekerom as the station closest to Veenkampen

# Add denitrification as a constant term (dNdt_den; ugN/h).
df_meteo['dNdt_den']        = 11.*14.*A*0.000001

# Initial conditions for the nutrients concentration.
C_N_pond_ini                = 3.0e-6      # ug/m3 Set initial N concentration in the pond (C_N_pond_ini; ug/m3).
mN_ini                      = C_N_pond_ini * V_ini

# Calculate lateral and vertical inflows of ammonia N.
df_meteo['dNdt_Ql_in']      = df_meteo['Ql_in']      * C_N_Ql_in # ug/h lateral  inflow
df_meteo['dNdt_Qk_in']      = df_meteo['Qk_in']      * C_N_Qk_in # ug/h vertical inflow

df_meteo['dNdt_Ql_in_real'] = df_meteo['Ql_in_real'] * C_N_Ql_in # ug/h lateral  inflow
df_meteo['dNdt_Qk_in_real'] = df_meteo['Qk_in_real'] * C_N_Qk_in # ug/h vertical inflow

### 6 Perform the time integration

1. Initialize the arrays for water balance calculation -> set up some values for volume and lateral outflow. These values will be later overwritten.
2. Define initial values (values of the first time step) for lateral outflow and volume.
3. Initialize the arrays for nutrient balance calculation.
4. Perform time integartion for A simple method where evaporation is constant, B method where evaporation is calculated using Makkink approach, and C method wheer evaporation is calculated using Makkink approach and Q-terms are realistic.

In [ ]:
# 1.---------------------------------------------------------------------------
# Initialisation of arrays for water balance calculation. 
# Note that these values will be rewritten during integration.
df_meteo['V'     ]                  = V_ini  # m3   Set up the water volume in the pond; Simple method
df_meteo['V_mak' ]                  = V_ini  # m3   Set up the water volume in the pond; Makkink method
df_meteo['V_real']                  = V_ini  # m3   Set up the water volume in the pond; Realistic method

df_meteo['Ql_out'     ]             = 0.     # m3/h Set up the lateral outflow;          Simple method
df_meteo['Ql_out_mak' ]             = 0.     # m3/h Set up the lateral outflow;          Makkink method
df_meteo['Ql_out_real']             = 0.     # m3/h Set up the lateral outflow;          Realistic method

# 2.---------------------------------------------------------------------------
# Define initial values for some changing terms in the water balance equation.
it                                  = df_meteo.index[0]
df_meteo.at[it, 'Ql_out'     ]      = 0.0    # m3/h Set up the initial lateral outflow; Simple method
df_meteo.at[it, 'Ql_out_mak' ]      = 0.0    # m3/h Set up the initial lateral outflow; Makkink method  
df_meteo.at[it, 'Ql_out_real']      = 0.0    # m3/h Set up the initial lateral outflow; Realistic method  

df_meteo.at[it, 'dV_dt'      ]      = 0.0    # m3/h Set up the initial volume tendency; Simple method
df_meteo.at[it, 'dV_dt_mak'  ]      = 0.0    # m3/h Set up the initial volume tendency; Makkink method
df_meteo.at[it, 'dV_dt_real' ]      = 0.0    # m3/h Set up the initial volume tendency; Realistic method

df_meteo.at[it, 'V'          ]      = V_ini  # m3   Set up the initial volume;          Simple method
df_meteo.at[it, 'V_mak'      ]      = V_ini  # m3   Set up the initial volume;          Makkink method
df_meteo.at[it, 'V_real'     ]      = V_ini  # m3   Set up the initial volume;          Realistic method

# 3.---------------------------------------------------------------------------
# Define initial values for some changing terms in the nutrient balance equation.
# Set the tendency terms to 0 ugN/h at the first timestep.
df_meteo.at[it, 'mN'              ] = mN_ini   # ug/m3  Initial N mass in the pond;            Simple method
df_meteo.at[it, 'mN_real'         ] = mN_ini   # ug/m3  Initial N mass in the pond;            Realistic method

df_meteo.at[it, 'dNdt_Ql_out'     ] = 0.       # ug N/h Tendency term due to lateral  outflow; Simple method 
df_meteo.at[it, 'dNdt_Ql_out_real'] = 0.       # ug N/h Tendency term due to lateral  outflow; Realistic method 

df_meteo.at[it, 'dNdt_Qk_out'     ] = 0.       # ug N/h Tendency term due to vertical outflow; Simple method
df_meteo.at[it, 'dNdt_Qk_out_real'] = 0.       # ug N/h Tendency term due to vertical outflow; Realistic method

df_meteo.at[it, 'dNdt_tot'        ] = 0.       # ug N/h Tendency term (total);                 Simple method
df_meteo.at[it, 'dNdt_tot_real'   ] = 0.       # ug N/h Tendency term (total);                 Realistic method

# 4.---------------------------------------------------------------------------
# Time integration. 
# First  we perform time integration to calculate volume when evaporation is constant. 
# Second we perform time integration to calculate volume when evaporation is changing using the Makkink method 
# which is noted using _mak. 
# Third we perform time integration to calculate volume when evaporation is changing using the Makkink method 
# and when lateral inflow and vertical in- and outflow are realistic which is noted using _real. 

# Start time loop (Note: the loop starts with the for ... command and all lines indented with 4 spaces belong to the for loop)
nt                                           = len(df_meteo)
for it in range(1,nt):
    it_now                                   = df_meteo.index[it  ]
    it_prev                                  = df_meteo.index[it-1]
    
    # A.--- Simple method, constant evaporation -----------------------------------------------------------------------
    # A.1 --- Water balance.
    df_meteo.at[it_now, 'Ql_out'     ]       = max(0., df_meteo.at[it_prev, 'V'] - V_max)
    df_meteo.at[it_now, 'dV_dt'      ]       = df_meteo.at[it_now,  'P'    ] - df_meteo.at[it_now, 'E'     ] \
                                             + df_meteo.at[it_now,  'Qk_in'] - df_meteo.at[it_now, 'Qk_out'] \
                                             + df_meteo.at[it_now,  'Ql_in'] - df_meteo.at[it_now, 'Ql_out']
    df_meteo.at[it_now, 'V'          ]       = df_meteo.at[it_prev, 'V'    ] + df_meteo.at[it_now, 'dV_dt']*dt     
    
    # A.2 --- Nutrient balance.
    C_N_pond                                 = df_meteo.at[  it_prev, 'mN']/df_meteo.at[it_prev,'V']
    df_meteo.at[it_now, 'dNdt_Ql_out']       = df_meteo.at[  it_now,  'Ql_out'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_Qk_out']       = df_meteo.at[  it_now,  'Qk_out'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_tot'   ]       = df_drydepo.at[it_now,  'dNdt_FNH3_dry'] + df_meteo.at[it_now, 'dNdt_FNH3_wet'] \
                                             + df_meteo.at[  it_now,  'dNdt_Ql_in'   ] - df_meteo.at[it_now, 'dNdt_Ql_out'  ] \
                                             + df_meteo.at[  it_now,  'dNdt_Qk_in'   ] - df_meteo.at[it_now, 'dNdt_Qk_out'  ] \
                                             - df_meteo.at[  it_now,  'dNdt_den'     ]
    df_meteo.at[it_now, 'mN'         ]       = df_meteo.at[  it_prev, 'mN']            + df_meteo.at[it_now, 'dNdt_tot']*dt    
    
    # B.--- Makkink method, changing evaporation --------------------------------------------------------------------- 
    # B.1 --- Water balance. (Note that Qlin, Qklin and Qkout remain identical to the simplistic case)
    df_meteo.at[it_now, 'Ql_out_mak' ]       = max(0., df_meteo.at[it_prev, 'V_mak'] - V_max)
    df_meteo.at[it_now, 'dV_dt_mak'  ]       = df_meteo.at[it_now, 'P'    ] - df_meteo.at[it_now, 'E_mak'     ] \
                                             + df_meteo.at[it_now, 'Qk_in'] - df_meteo.at[it_now, 'Qk_out'    ] \
                                             + df_meteo.at[it_now, 'Ql_in'] - df_meteo.at[it_now, 'Ql_out_mak']
    df_meteo.at[it_now, 'V_mak'      ]       = df_meteo.at[it_prev,'V_mak'] + df_meteo.at[it_now, 'dV_dt_mak' ]*dt  

    # B.2 --- Nutrients (Note that FNH3_dry and _wet, dNdt_Ql_in, dNdt_Qk_in, dNdt_Qk_in and dNdt_den remain identical to the simplistic case)
    C_N_pond                                 = df_meteo.at[it_prev, 'mN_real']/df_meteo.at[it_prev,'V_real']
    df_meteo.at[it_now, 'dNdt_Ql_out_mak' ]  = df_meteo.at[ it_now, 'Ql_out_mak'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_tot_mak'    ]  = df_drydepo.at[it_now,'dNdt_FNH3_dry'  ] + df_meteo.at[it_now, 'dNdt_FNH3_wet'  ] \
                                             + df_meteo.at[ it_now, 'dNdt_Ql_in'     ] - df_meteo.at[it_now, 'dNdt_Ql_out_mak'] \
                                             + df_meteo.at[ it_now, 'dNdt_Qk_in'     ] - df_meteo.at[it_now, 'dNdt_Qk_out'    ] \
                                             - df_meteo.at[ it_now, 'dNdt_den'       ]
    df_meteo.at[it_now, 'mN_real'         ]  = df_meteo.at[ it_prev,'mN_real']         + df_meteo.at[it_now, 'dNdt_tot_mak']*dt

    
    # C.--- Realistic method, evaporation changing using Makkink, Q-terms are realistic. -----------------------------
    # C.1 --- Water balance.
    df_meteo.at[it_now, 'Ql_out_real']       = max(0., df_meteo.at[it_prev, 'V_real'] - V_max)
    df_meteo.at[it_now, 'dV_dt_real' ]       = df_meteo.at[it_now, 'P'         ] - df_meteo.at[it_now, 'E_mak'      ] \
                                             + df_meteo.at[it_now, 'Qk_in_real'] - df_meteo.at[it_now, 'Qk_out_real'] \
                                             + df_meteo.at[it_now, 'Ql_in_real'] - df_meteo.at[it_now, 'Ql_out_real']
    df_meteo.at[it_now, 'V_real'     ]       = df_meteo.at[it_prev,'V_real'    ] + df_meteo.at[it_now, 'dV_dt_real' ]*dt  

    # C.2 Nutrient balance.
    C_N_pond                                 = df_meteo.at[it_prev, 'mN_real']/df_meteo.at[it_prev,'V_real']
    df_meteo.at[it_now, 'dNdt_Ql_out_real']  = df_meteo.at[ it_now, 'Ql_out_real'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_Qk_out_real']  = df_meteo.at[ it_now, 'Qk_out_real'] * C_N_pond
    df_meteo.at[it_now, 'dNdt_tot_real'   ]  = df_drydepo.at[it_now, 'dNdt_FNH3_dry' ] + df_meteo.at[it_now, 'dNdt_FNH3_wet'   ] \
                                             + df_meteo.at[ it_now, 'dNdt_Ql_in_real'] - df_meteo.at[it_now, 'dNdt_Ql_out_real'] \
                                             + df_meteo.at[ it_now, 'dNdt_Qk_in_real'] - df_meteo.at[it_now, 'dNdt_Qk_out_real'] \
                                             - df_meteo.at[ it_now, 'dNdt_den'       ]
    df_meteo.at[it_now, 'mN_real'         ]  = df_meteo.at[ it_prev,'mN_real']         + df_meteo.at[it_now, 'dNdt_tot_real']*dt
    
    # D.--- Sensitivity study
    # You can now copy method C to below here and perform any sensitivity study that is necessary to better understand the model.
    # ADD ....
    
#<-> End time loop    
    


In [ ]:
# Calculate chlorofyl mass in algae in the pond and visibility

Chlorofyl_ini     = 0.1        # mg/l 
df_meteo['Chlorofyl']  = Chlorofyl_ini

# Parameter settings
r                 = 0.5             # Growth rate
l                 = 0.005           # Loss rate
Tmin              = 0.0             # oC Min temperature for algae growth
Topt              = 20.0            # oC Optimal temperature
Tmax              = 40.0            # oC Max temperature for algae growth
hN                = 0.40            # mg/l Sensitivity of algae growth to N concentration
hP                = 0.15            # mg/l Sensitivity of algae growth to P concentration
ha                = 15.             # Crowding factor
depth             = 1.5             # m Depth in m

# Temperature limitation
Ta                = df_meteo['Ta'].copy()
Tlim              = Ta * 0.0        # init
Tlim[(Ta >  Tmin) & (Ta < Topt)] = Ta/Topt
Tlim[(Ta >= Topt) & (Ta < Tmax)] = 1-(Ta-Topt)/Topt

# Nitrogen limitation
Rg                = df_meteo['Rg'].copy()
Llim              = Rg * 0.0        # init
Llim[Rg >  800]   = 1.0
Llim[Rg <= 800]   = Rg / 800.

# Light limitation
C_N               = df_meteo['mN_real'] / df_meteo['V_real'] # ug N /m3 Use 'REAL' scenario or change to another one
C_N               = C_N *1.0e6   # * from_ugperm3_to_mgperl
Nlim              = C_N  / (hN + C_N)

# Phosphorus limitation
C_P               = 5.0              # ug/l (fixed value for now, adapt if you have better measurements)
Plim              = C_P / (hP + C_P)

# Nutrient limitation
Nutlim            = np.minimum(Nlim,Plim)

nt                                           = len(df_meteo)
for it in range(1,nt):
    it_now                                   = df_meteo.index[it  ]
    it_prev                                  = df_meteo.index[it-1]

    #--- Chlorofyl content in algae. --------------------------------------------------------------------------
    df_meteo.at[it_now, 'Chlorofyl']         = df_meteo.at[it_prev, 'Chlorofyl'] + (r * df_meteo.at[it_prev,'Chlorofyl'] \
                                             * Llim.at[it_now] * Tlim[it_now] * Nutlim[it_now]                           \
                                             * (ha/(ha+df_meteo.at[it_prev, 'Chlorofyl'])))                              \
                                             -  l *    df_meteo.at[it_prev, 'Chlorofyl']
#<-> End time loop

# F. Calculate Secchi depth based on chlorofyl content (diagnostically, does not need to be included in the time loop)
df_meteo['Secchi'] = np.minimum(10**(0.65-0.43*np.log10(df_meteo['Chlorofyl'])), depth)

### 7 Make plots 

In [ ]:
# Plot 1:-----------------------------------------------------------
# Let's save the data into the same matrixfor easier plotting.
df_volume = pd.concat([df_meteo['V'], df_meteo['V_mak'], df_meteo['V_real']], axis=1, sort=False)
df_volume.columns = ['V', 'V_mak', 'V_real']

# Plot the time series of calculated water volume. 
# Plot Time [year] on x-axis and Volume [m3] on y-axis. 
# Note that for easier recogniotion we divided the volume by 1000. 
fig1 = (df_volume/1000).iplot(asFigure=True, xTitle='Time [year]', yTitle='Volume [1e3 m3]', width=2) 
fig1.show()    




In [ ]:
# Plot 2:-----------------------------------------------------------
# Plot the time series of nitrogen concentration. 
# Plot Time [year] on x-axis and Concentration [ug m^-3] on y-axis. 
C_N_pond      = df_meteo['mN'     ]/df_meteo['V'     ]
C_N_pond_real = df_meteo['mN_real']/df_meteo['V_real']
df_C_N = pd.concat([C_N_pond, C_N_pond_real],axis=1, sort=False)
df_C_N.columns = ['Simplistic', 'Realistic']

fig2 = (df_C_N).iplot(asFigure=True, xTitle='Time [year]', yTitle='Concentration [ug/m3]', width=2) 
fig2.show()


In [ ]:
# Plot 3:-----------------------------------------------------------
# Plot the time series of chlorofyl and secchi.
fig3 = df_meteo[['Chlorofyl', 'Secchi']].iplot(asFigure=True, subplots=True, shape=(2,1), shared_xaxes=True,
    layout=dict(yaxis=dict(title='Chlorofyl [ug/l]'), xaxis=dict(title=' ')))
fig3['layout']['yaxis2'].update({'title':'Secchi [m]'})
fig3['layout']['xaxis2'].update({'title':'Time [years]'})
fig3.show()

### 8 Save the data
You may save the data to a csv file to download. But we recommend to try and use our example scripts to make the figures online in the notebook. This is because writing the figures as scripts is a lot easier if you change any setting in the model. In the notebook, you just rerun the figure script. In Excel you need to download the data, import it in Excel and make the figure all over again.

In [ ]:
df_meteo.to_csv('../Data_IntegrationCourseSWA/Data_IntegrationSkills_PBL_FinalOutput.csv', sep=';')

### 9 Close the notebook

Now, to limit the data storage please do the following:
- go to *KERNEL* --> *RESTART and CLEAR OUTPUT*. This removes the interactive graphs from your notebook, but leaves your answers intact.
- go to *FILE* --> *SAVE and CHECKPOINT*. This saves your answers in the notebook
- go to *FILE* --> *CLOSE and HALT*, to shutdown the notebook. You can always come back later and restart.